# Define Functions and Run Initial Query

In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import itertools

pd.options.display.float_format = '{:,.5f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 50

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df = df.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    TestGroup = np.array(('364717','426097','547655','552121',
        '554487','569375','572247','572815',
        '572909','573190','573192','573276',
        '573573','573585','575731','575996',
        '576565','576911','577073','577246',
        '579162','580895','580911','581015',
        '581139','581145','581275')).astype(int)
    df['TestGroup'] = (df['Employee'].isin(TestGroup)).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    df['Weeks Ago'] = df['Week'].astype(int).max() - df['Week'].astype(int)
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    df['Constant'] =1
    return df

def add_metrics(df):
    df = df.copy()
    df['AHT'] = df['Total_Resolution_Time']/df['Sessions']
    df['HERO'] = df['hero_solve_score']/df['hero_solve_count']
    df['SP100'] = df['Total_Accepted']/df['Total_Eligible']
    df['Constant'] =1
    return df

In [ ]:
dfp = run_query(file_to_string('DailyMetrics.SQL'))

In [ ]:
display(dfp)

# Test
1. Creates representative samples
2. Tests those samples if they are within a number of standard deviations from the test group. If so adds them to a list for further examination.

In [ ]:
experts = add_conditionals(dfp.copy())
experts = experts[['Employee','Supervisor','TestGroup']]
experts= experts.drop_duplicates()
experts= experts[experts['Employee'].isin([
    548026,548646,550446,578299,552402,567519,568568,574166,581396,581694,293932,536491,570227,569153,579630,549667,547655,552121,569375,572247,572815,572909,573190,573192,573276,573573,573585,575996,576911,581139
])]

# Define Metrics and Groups Being Tested
mainMetrics=['Sessions','AHT','SP100','HERO']
testGroup = ['Expert Assist Members','Proactive EA Members']


# Get yearly metrics
results_pivot = pd.pivot_table(dfp.copy(),index='Employee',values=['Total_Accepted','Total_Eligible','hero_solve_score','hero_solve_count','Total_Resolution_Time','Sessions'],aggfunc=np.sum)
results_pivot = add_metrics(results_pivot)
results_pivot = results_pivot[mainMetrics]

for metric in mainMetrics:
    results_pivot[metric] = (results_pivot[metric]-results_pivot[metric].mean())/results_pivot[metric].std()
    del metric
# Carve out members of control group to get proactive
control = experts[experts['TestGroup']==0].copy()
pilot_coaches = ['joyce herrera', 'valarie hutchinson']
control_to_reactive = control[control['Supervisor'].isin(pilot_coaches)].copy()['Employee']
control = control[~control['Supervisor'].isin(pilot_coaches)]['Employee']
# print(len(control))
# Get series of experts currently in test group
expert_assist_experts = experts[experts['TestGroup']==1].copy()['Employee']
# Build out samples in testFrame

testFrame = pd.DataFrame(columns={testGroup[0]:[],testGroup[0]:[]})
del testGroup
test_samples = list(itertools.combinations(expert_assist_experts,r=10)) 

for sample in test_samples:
    eaList = list(expert_assist_experts[~expert_assist_experts.isin(sample)])
    # display(eaList)
    eaList = eaList+control_to_reactive.to_list()
    # display(eaList)
    prList = list(pd.concat([pd.Series(sample)]))
    eaResults=results_pivot[results_pivot.index.isin(eaList)].mean()
    prResults=results_pivot[results_pivot.index.isin(prList)].mean()
    appendFrame =pd.DataFrame(data={
        'Expert Assist Members': [eaList],
        'Proactive EA Members':[prList],
        'Expert Assist Members Sessions':eaResults['Sessions'],
        'Expert Assist Members AHT':eaResults['AHT'],
        'Expert Assist Members SP100':eaResults['SP100'],
        'Expert Assist Members HERO':eaResults['HERO'],
        'Proactive EA Members Sessions':prResults['Sessions'],
        'Proactive EA Members AHT':prResults['AHT'],
        'Proactive EA Members SP100':prResults['SP100'],
        'Proactive EA Members HERO':prResults['HERO']
        })
    testFrame=pd.concat([testFrame,appendFrame],axis=0)
    del appendFrame, eaResults, prResults, eaList, prList, sample,
del test_samples
testFrame.to_excel('ExpertGroupings.xlsx')
    
display(len(testFrame['Expert Assist Members'].iloc[0]),
    len(testFrame['Proactive EA Members'].iloc[0]))
display(len(control),list(control),pd.DataFrame(results_pivot[results_pivot.index.isin(control)].mean()))
del control, control_to_reactive,  experts, expert_assist_experts, pilot_coaches,results_pivot, testFrame, mainMetrics

In [ ]:
df =dfp.copy()
df['Date']=pd.to_datetime(df['Date'])
pivot = df.copy()
pivot_base = pivot[pivot['Sessions']>=0]
pivot = pd.pivot_table(pivot_base.copy(),aggfunc=max,index='Employee',values='Date').sort_values('Date')
pivot = pd.concat((pivot,pd.pivot_table(pivot_base.copy(),aggfunc=sum,index='Employee',values='Sessions')),axis=1).sort_values('Date')
px.bar(pd.pivot_table(df[df['Employee'].isin([549667,573190,293932,536491])].copy(),index='Date',values='Sessions',columns='Employee',aggfunc=sum)).show()
px.line(pd.pivot_table(df.copy(),index='Date',values='Sessions',columns='Employee')).show()
display(len(pivot),pivot)
del df, pivot_base, pivot